In [ ]:
import pandas as pd
import networkx as nx
from scipy.spatial import KDTree
import matplotlib.pyplot as plt
from tqdm import tqdm
from joblib import Parallel, delayed
from folium.plugins import TimestampedGeoJson
from geopy.distance import geodesic
from sklearn.cluster import DBSCAN
from sklearn.preprocessing import StandardScaler
from pandas import DataFrame
import time
import numpy as np
import folium

## Useful support function

In [ ]:
def create_plot(data):
    # Ambil baris pertama dari DataFrame
    first_row = data.iloc[0]

    # Ambil nilai latitude dan longitude dari baris pertama
    latitude = first_row['latitude']
    longitude = first_row['longitude']
    
    m = folium.Map(location=[latitude, longitude], zoom_start=25)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"User ID: {row['maid']}<br>Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [ ]:
def create_plot_raw(data):
    # Ambil baris pertama dari DataFrame
    first_row = data.iloc[0]

    # Ambil nilai latitude dan longitude dari baris pertama
    latitude = first_row['latitude']
    longitude = first_row['longitude']
    
    m = folium.Map(location=[latitude, longitude], zoom_start=25)

    # Add CircleMarkers for each point
    for index, row in data.iterrows():
        folium.CircleMarker(
            location=[row["latitude"], row["longitude"]],
            radius=5,  # Marker size
            color="blue",  # Marker color
            fill=True,
            fill_color="blue",  # Fill color of the marker
            fill_opacity=0.7,  # Opacity of the marker fill
            popup=f"Latitude: {row['latitude']}<br>Longitude: {row['longitude']}",
        ).add_to(m)
    
    return m

In [ ]:
def create_pivot(df):
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    df_pivot = df['maid'].groupby(df['timestamp'].dt.date).value_counts()
    pivot = df_pivot.unstack().fillna(0).astype(int)

    total_counts = pivot.sum(axis=0)
    sorted_columns = total_counts.sort_values(ascending=False).index
    pivot_sorted = pivot[sorted_columns]
    return pivot_sorted

In [ ]:
def create_line(data):
    data = data.to_crs('EPSG:4326')
    data = data['geometry']
    # Buat peta dengan lokasi awal berdasarkan rata-rata koordinat dari data linestring
    avg_lat = data.apply(lambda x: x.centroid.y).mean()
    avg_lon = data.apply(lambda x: x.centroid.x).mean()
    m = folium.Map(location=[avg_lat, avg_lon], zoom_start=15)

    # Tambahkan polyline untuk setiap linestring
    for linestring in data:
        coordinates = [(lat, lon) for lon, lat in linestring.coords]
        folium.PolyLine(locations=coordinates, color='purple', weight=8, opacity=0.7).add_to(m)

    # Tambahkan GeoJson dari data GeoPandas
    folium.GeoJson(data.to_json(), name='Garis Jalan').add_to(m)

    return m

## 1. Filter Area around Malioboro

In [ ]:
def filter_area(gps, road):
    road_path = road
    
    tdf = skmob.TrajDataFrame.from_file(gps)
    area_shape = gpd.read_file(road_path)
    
    gdf_gps = GeoDataFrame(tdf, geometry=gpd.points_from_xy(tdf['longitude'], tdf['latitude']))
    filtered_data = gdf_gps[gdf_gps.geometry.within(area_shape.geometry.iloc[0])].copy()
    
    filtered_data['datetime_wib'] = pd.to_datetime(filtered_data['datetime_wib'])
    filtered_data['tanggal'] = filtered_data['datetime_wib'].dt.date
    
    print("Filtering area succeed")
    
    return filtered_data

In [ ]:
# Read reverse geocoding data
gps = '../../DataGPS/RGdesember2021.csv'
road = './Malioboro_around/Shrink/clipping_boundary.geojson'

filtered_data = filter_area(gps, road)
filtered_data.to_csv('filter1_malas_des.csv', index=False)    


## 2. Preprocessing basic
- Changed datetime_wib column -> timestamp
- Convert data type timestamp to datetime
- Get the required columns df[['maid', 'latitude', 'longitude', 'timestamp']]
- Remove duplicate data based on the same maid and time
- Sort data by maid and timestamp

In [ ]:
def preprocess_gps_data(df):
    # Step 1: Ubah nama kolom datetime_wib menjadi timestamp
    df = df.rename(columns={'datetime_wib':'timestamp'})
    
    # Step 2: Konversi tipe data pada kolom timestamp menjadi datetime
    df['timestamp'] = pd.to_datetime(df['timestamp'])
    
    # Step 3: Ambil hanya kolom maid, latitude, longitude, dan timestamp
    df_filtered = df[['maid', 'latitude', 'longitude', 'timestamp']]
        
    # Step 4: Hapus data duplikat utk data keseluruhan dan data duplikat utk maid dan timestamp yg sama 
    # (ga mgkn berada pada dua tmpt berbeda dalam satu waktu kan)
    df_filtered = df_filtered.drop_duplicates(subset=['maid', 'timestamp'])
    
    # Step 5: Hapus data yang hanya memiliki 1 record data dalam 1 hari
    df_filtered['date'] = df_filtered['timestamp'].dt.date
    record_count_per_day = df_filtered.groupby(['maid', 'date']).size().reset_index(name='count')
    multiple_records_per_day = record_count_per_day[record_count_per_day['count'] > 1]
    df_filtered = pd.merge(df_filtered, multiple_records_per_day[['maid', 'date']], on=['maid', 'date'], how='inner')
    df_filtered.drop('date', axis=1, inplace=True)

    # Step 6: Urutkan data berdasarkan maid dan timestamp
    df_filtered = df_filtered.sort_values(by=['maid', 'timestamp'])
    
    # Step 7: Reset index
    df_filtered.reset_index(inplace=True, drop=True)
    
    return df_filtered

In [ ]:
df_preprocessed = preprocess_gps_data(filtered_data)
df_preprocessed.to_csv('filter2_malar_jan.csv', index=False)

## 3. Filter data points with radius <20m from the road network

In [ ]:
def filter_area_around_road(gps_data, road_data, distance):
    # Membuat GeoDataFrame dari data GPS
    gdf_gps = gpd.GeoDataFrame(
        gps_data,
        geometry=gpd.points_from_xy(gps_data.longitude, gps_data.latitude),
        crs="EPSG:4326"  # WGS84 coordinate reference system
    )

    # Membuat GeoDataFrame dari data jaringan jalan
    gdf_jalan = gpd.GeoDataFrame(
        road_data,
        geometry=road_data.geometry,
        crs="EPSG:4326"  # WGS84 coordinate reference system
    )

    # Mengubah sistem referensi koordinat jaringan jalan ke UTM (misalnya UTM zone 48S untuk Yogyakarta)
    gdf_jalan = gdf_jalan.to_crs("EPSG:32748")  # UTM zone 48S

    # Buffer jaringan jalan sebesar 20 meter
    buffered_jalan = gdf_jalan.buffer(distance)

    # Gabungkan semua buffered jalan menjadi satu geometri
    merged_buffer = buffered_jalan.unary_union

    # Mengubah sistem referensi koordinat data GPS ke UTM
    gdf_gps = gdf_gps.to_crs("EPSG:32748")  # UTM zone 48S

    # Membuat kolom baru untuk menyimpan jarak terdekat ke jalan
    gdf_gps['closest_distance_to_road'] = gdf_gps.geometry.apply(lambda x: x.distance(merged_buffer))

    # Mengambil data point yang jaraknya maksimal 20 meter dari jaringan jalan
    data_point_terdekat = gdf_gps[gdf_gps['closest_distance_to_road'] <= distance]

    return data_point_terdekat

In [ ]:
# Read the road data
road = './Malioboro_around/Around_Malioboro_line.shp'
area_shape = gpd.read_file(road)

highway = ['living_street','residential','primary','primary_link','tertiary', 'secondary', 'unclassified', 'secondary_link', 'tertiary_link','track']
area_vehicle = area_shape[area_shape['highway'].isin(highway)]
area_vehicle = area_vehicle[['osm_id','oneway','name', 'highway', 'geometry']]
area_vehicle.reset_index(inplace=True, drop=True)

In [ ]:
# Do the filering process
gps_data = df_preprocessed
road_data = area_vehicle
df_filtered_5m = filter_area_around_road(gps_data, road_data, 5)

## 4. Map matching with KDTree to mapping all gps points to the road

In [1]:
# For trial with small data
data = pd.read_csv('../../DataGPS_MalioboroAround/filter2_malar_des.csv')
data['timestamp'] = pd.to_datetime(data['timestamp'])
gps = data[(data['maid'] == 'd9f515f4-1a78-442b-81c4-55bf5f253b91') & (data['timestamp'].dt.date == pd.to_datetime('2021-12-06').date())]
gps.reset_index(inplace=True, drop=True)

NameError: name 'pd' is not defined

In [2]:
# Pass the dataframe from the previous process
gps = df_filtered_5m

In [ ]:
# Reading the road CSV file into a DataFrame
road = pd.read_csv('./expanded_road_df_DIY.csv')

,Unnamed: 0,name,start_x,start_y,end_x,end_y
0,0,NaN,110.145676,-7.652724,110.145675,-7.652734
1,1,NaN,110.145676,-7.652724,110.145679,-7.652716
2,2,NaN,110.398799,-7.861408,110.398598,-7.861393
3,3,NaN,110.398799,-7.861408,110.398927,-7.861439
4,4,NaN,110.581033,-7.788455,110.580238,-7.788295
...,...,...,...,...,...,...
962264,962264,NaN,110.576041,-7.856010,110.576043,-7.856035
962265,962265,NaN,110.448121,-7.898401,110.448103,-7.898251
962266,962266,NaN,110.290861,-8.012311,110.290827,-8.012374
962267,962267,NaN,110.328035,-7.766726,110.328073,-7.766544


In [ ]:
# Create Graph from Road Network Data
G = nx.Graph()
for index, row in road.iterrows():
    G.add_edge((row['start_x'], row['start_y']), (row['end_x'], row['end_y']), weight=1, name=row['name'])

In [ ]:
# Map Matching with KDTree untuk menarik data yg tidak di jalan menjadi berada di jalan

sorted_gps = gps.sort_values(by=['maid', 'timestamp'])
node_list = list(G.nodes())

tree = KDTree(node_list)
distances, indices = tree.query(sorted_gps[['longitude', 'latitude']].values)
all_mapped_points_kdtree = [node_list[index] for index in indices]

sorted_gps['adjusted_longitude'] = [point[0] for point in all_mapped_points_kdtree]
sorted_gps['adjusted_latitude'] = [point[1] for point in all_mapped_points_kdtree]
sorted_gps['timestamp'] = pd.to_datetime(sorted_gps['timestamp'])

## 5. Filter immobility data with mas Arkham algorithm

## 6. Adjusted path also based on mas Arkham algorithm

## 7. Split trajectories based on observation data

In [ ]:
import movingpandas as mpd
import shapely as shp
import hvplot.pandas 
import matplotlib.pyplot as plt
import geoviews
import folium
import mapclassify

from geopandas import GeoDataFrame, read_file
from shapely.geometry import Point, LineString, Polygon
from datetime import datetime, timedelta
from holoviews import opts

import warnings
warnings.filterwarnings('ignore')

plot_defaults = {'linewidth':5, 'capstyle':'round', 'figsize':(9,3), 'legend':True}
opts.defaults(opts.Overlay(active_tools=['wheel_zoom'], frame_width=500, frame_height=400))

In [ ]:
data_mpd = adjusted_gps_data
tdf = mpd.TrajectoryCollection(data_mpd, traj_id_col='maid', t='timestamp', y='latitude', x='longitude')
tdf

TrajectoryCollection with 1 trajectories

In [ ]:
def split_trajectories(tdf, minute_gap):
    index = 0
    df_min = pd.DataFrame()

    while index < len(tdf):
        my_traj = tdf.trajectories[index]
        split = mpd.ObservationGapSplitter(my_traj).split(gap=timedelta(minutes=minute_gap))
        if len(split) > 1:
            split_df = split.to_traj_gdf()
            df_min = df_min._append(split_df, ignore_index=True)
        index += 1

    return df_min

In [ ]:
def create_split_traj(split_df, adjusted_gps_data):
    split_gps_data = adjusted_gps_data.copy()

    for index, row in split_df.iterrows():
        start_t = row['start_t']
        end_t = row['end_t']
        correlated_string = row['maid'].split('_')[-1]  # Get last two strings after "_"
        mask = (adjusted_gps_data['timestamp'] >= start_t) & (adjusted_gps_data['timestamp'] <= end_t)
        split_gps_data.loc[mask, 'maid'] = split_gps_data.loc[mask, 'maid'] + '_' + correlated_string
    
    return split_gps_data   

In [ ]:
minute_gap = 15 #Change this variable to determine the observation timedelta gap
split_gps_data = create_split_traj(split_trajectories(tdf, minute_gap), adjusted_gps_data)
# split_gps_data.to_csv(f"split_gps_data_{minute_gap}.csv", index=False)

In [ ]:
split_gps_data

,maid,latitude,longitude,timestamp,adjusted_latitude,adjusted_longitude
0,d9f515f4-1a78-442b-81c4-55bf5f253b91_0,-7.777150,110.367330,2021-12-06 08:45:05,-7.777179,110.367552
1,d9f515f4-1a78-442b-81c4-55bf5f253b91_0,-7.777140,110.367340,2021-12-06 08:45:50,-7.777168,110.367553
2,d9f515f4-1a78-442b-81c4-55bf5f253b91_0,-7.778760,110.373337,2021-12-06 08:46:34,-7.778741,110.373283
3,d9f515f4-1a78-442b-81c4-55bf5f253b91_0,-7.778730,110.373320,2021-12-06 08:46:43,-7.778721,110.373288
4,d9f515f4-1a78-442b-81c4-55bf5f253b91_0,-7.779930,110.373060,2021-12-06 08:47:08,-7.779912,110.372994
...,...,...,...,...,...,...
315,d9f515f4-1a78-442b-81c4-55bf5f253b91_6,-7.777660,110.375780,2021-12-06 17:45:54,-7.777660,110.375782
316,d9f515f4-1a78-442b-81c4-55bf5f253b91_6,-7.776130,110.374847,2021-12-06 17:46:07,-7.776121,110.374848
317,d9f515f4-1a78-442b-81c4-55bf5f253b91_6,-7.776140,110.374863,2021-12-06 17:46:57,-7.776122,110.374863
318,d9f515f4-1a78-442b-81c4-55bf5f253b91_6,-7.773060,110.375664,2021-12-06 17:47:07,-7.773045,110.375629


## 8. Path reconstruction with PyTrack

from pytrack.graph import graph, distance, download
from pytrack.analytics import visualization
from pytrack.matching import candidate, mpmatching_utils, mpmatching

In [ ]:
def map_matching_pytrack(group, graph):
    df = group
    G = graph
    maid = df['maid'].values[0]

    latitude = df["adjusted_latitude"].to_list()
    longitude = df["adjusted_longitude"].to_list()
    points = [(lat, lon) for lat, lon in zip(latitude, longitude)]

    # Extract candidates
    G_interp, candidates = candidate.get_candidates(G, points, interp_dist=10, closest=True, radius=30)

    # Extract trellis DAG graph
    trellis = mpmatching_utils.create_trellis(candidates)

    # Perform the map-matching process
    path_prob, predecessor = mpmatching.viterbi_search(G_interp, trellis, "start", "target")

    path_df = pd.DataFrame()
    bad_df = pd.DataFrame()

    # Extract the path results from map-matching process
    if len(predecessor) > 0:
        node, path = mpmatching_utils.create_matched_path(G_interp, trellis, predecessor)

        # Create constructed path
        path_df = pd.DataFrame(path)
        path_df = path_df.rename(columns={0:'latitude', 1:'longitude'})
        path_df.insert(loc=0, column='maid', value=maid)

    else:
        bad_df = bad_df._append({'bad_maid': maid}, ignore_index=True)

    return path_df, bad_df

In [ ]:
df = split_gps_data

In [ ]:
latitude = df["adjusted_latitude"].to_list()
longitude = df["adjusted_longitude"].to_list()

points = [(lat, lon) for lat, lon in zip(latitude, longitude)]
north, east = np.max(np.array([*points]), 0)
south, west = np.min(np.array([*points]), 0)

In [ ]:
G = graph.graph_from_bbox(*distance.enlarge_bbox(north, south, west, east, 300), simplify=True, network_type='drive')

Downloaded 1,281.01kB


In [ ]:
def urutkan_maid(df, maid):
    # Ekstrak angka terakhir dari setiap elemen dalam kolom 'bad_maid'
    df['last_number'] = df[maid].str.extract(r'_(\d+)$')

    # Ubah angka terakhir menjadi tipe data numerik
    df['last_number'] = pd.to_numeric(df['last_number'])

    # Urutkan DataFrame berdasarkan angka terakhir secara numerik
    df = df.sort_values(by='last_number', ignore_index=True)

    # Hapus kolom 'last_number' jika tidak diperlukan lagi
    df.drop(columns='last_number', inplace=True)

    return df

In [ ]:
# Group by trajectories to perform Map Matching with PyTrack
def map_matching_pytrack_parallel(data, G):
    grouped = data.groupby('maid')
    results, bad_paths = zip(*(Parallel(n_jobs=40)(delayed(map_matching_pytrack)(group, G) for _, group in tqdm(grouped))))
    matched_path = urutkan_maid(pd.concat(results, ignore_index=True), 'maid')
    bad_path = urutkan_maid(pd.concat(bad_paths, ignore_index=True), 'bad_maid')
    
    return matched_path, bad_path

In [ ]:
# Perform the map matching process for all maid
matched_path, bad_path = map_matching_pytrack_parallel(df, G)

## 9. Time interpolation

In [ ]:
from scipy.spatial import distance_matrix
def find_nearest_point_index(lat, lon, points_df):
    distances = distance_matrix(points_df[['latitude', 'longitude']], [(lat, lon)])
    nearest_point_index = np.argmin(distances)
    return nearest_point_index

In [ ]:
def add_timestamp_to_nearest_point(path_df, df):
    nearest_points_timestamp = {}  # Dictionary to store timestamps for each nearest point index
    for index, row in df.iterrows():
        nearest_point_index = find_nearest_point_index(row['adjusted_latitude'], row['adjusted_longitude'], path_df)
        if nearest_point_index not in nearest_points_timestamp:
            nearest_points_timestamp[nearest_point_index] = row['timestamp']
        else:
            # If multiple points share the same nearest point index, choose the latest timestamp
            nearest_points_timestamp[nearest_point_index] = max(nearest_points_timestamp[nearest_point_index], row['timestamp'])
    
    # Assign timestamps to the nearest points in path_df
    path_df['timestamp'] = path_df.index.map(lambda idx: nearest_points_timestamp.get(idx))
    
    return path_df

In [ ]:
df  = pd.read_csv("df.csv")
path_df_30persen = pd.read_csv("path_df_30persen.csv")
path_df_60persen = path_df_30persen[::2]
hasil = add_timestamp_to_nearest_point(path_df_60persen, df)
hasil.reset_index(inplace=False, drop=True)
hasil[:20]

C:\Users\cengceng\AppData\Local\Temp\ipykernel_90480\3938065002.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  path_df['timestamp'] = path_df.index.map(lambda idx: nearest_points_timestamp.get(idx))


,latitude,longitude,timestamp
0,-7.785370,110.374630,2021-12-06 11:52:43
2,-7.785877,110.374540,None
4,-7.786409,110.374447,None
6,-7.786876,110.374521,None
8,-7.787103,110.375029,2021-12-06 11:53:43
10,-7.787559,110.375270,None
12,-7.788052,110.375040,None
14,-7.788450,110.374748,None
16,-7.788567,110.374238,2021-12-06 11:54:40
18,-7.788788,110.373916,None
